CREATE DIR STRUCTURE FOR ANIPOSE

In [1]:
import os
import toml

In [2]:
# choose project name and working directory
new_project_directory = r'C:/Users/myore/Desktop/MYOREHAB'
new_project_name = 'HandTracking'

# get list of new sessions
sessionlist = [
    'session1',
    'session2',
    ]

In [3]:
# create project structure
for session in sessionlist:
    # make videos-raw folder for behavior
    videopath = os.path.join(new_project_directory, new_project_name, session, 'videos-raw')
    os.makedirs(videopath)

    # make calibration folder
    calibrationpath = os.path.join(new_project_directory, new_project_name, session, 'calibration')
    os.makedirs(calibrationpath)

# save project directory
projectpath = os.path.join(new_project_directory, new_project_name)
os.chdir(projectpath)

print(f'Done! New Anipose Project created: {os.getcwd()}')

Done! New Anipose Project created: C:\Users\myore\Desktop\MYOREHAB\HandTracking


CREATE config.toml file

In [4]:
# default from anipos: https://github.com/lambdaloop/anipose/blob/master/anipose/anipose.py
DEFAULT_CONFIG = {
    'video_extension': 'mp4',
    'converted_video_speed': 1,
    'calibration': {
        'animal_calibration': False,
        'calibration_init': None,
        'fisheye': False
    },
    'manual_verification': {
        'manually_verify': False
    },
    'triangulation': {
        'ransac': False,
        'optim': False,
        'scale_smooth': 2,
        'scale_length': 2,
        'scale_length_weak': 1,
        'reproj_error_threshold': 5,
        'score_threshold': 0.8,
        'n_deriv_smooth': 3,
        'constraints': [],
        'constraints_weak': []
    },
    'pipeline': {
        'videos_raw': 'videos-raw',
        'videos_raw_mp4': 'videos-raw-mp4',
        'pose_2d': 'pose-2d',
        'pose_2d_filter': 'pose-2d-filtered',
        'pose_2d_projected': 'pose-2d-proj',
        'pose_3d': 'pose-3d',
        'pose_3d_filter': 'pose-3d-filtered',
        'videos_labeled_2d': 'videos-labeled',
        'videos_labeled_2d_filter': 'videos-labeled-filtered',
        'calibration_videos': 'calibration',
        'calibration_results': 'calibration',
        'videos_labeled_3d': 'videos-3d',
        'videos_labeled_3d_filter': 'videos-3d-filtered',
        'angles': 'angles',
        'summaries': 'summaries',
        'videos_combined': 'videos-combined',
        'videos_compare': 'videos-compare',
        'videos_2d_projected': 'videos-2d-proj',
    },
    'filter': {
        'enabled': False,
        'type': 'medfilt',
        'medfilt': 13,
        'offset_threshold': 25,
        'score_threshold': 0.05,
        'spline': True,
        'n_back': 5,
        'multiprocessing': False
    },
    'filter3d': {
        'enabled': False
    }
}

# create config.toml file
configfile = os.path.join(projectpath, 'config.toml')

with open(configfile, 'w') as f:
     new_toml_string = toml.dump(DEFAULT_CONFIG, f)
print(new_toml_string)

video_extension = "mp4"
converted_video_speed = 1

[calibration]
animal_calibration = false
fisheye = false

[manual_verification]
manually_verify = false

[triangulation]
ransac = false
optim = false
scale_smooth = 2
scale_length = 2
scale_length_weak = 1
reproj_error_threshold = 5
score_threshold = 0.8
n_deriv_smooth = 3
constraints = []
constraints_weak = []

[pipeline]
videos_raw = "videos-raw"
videos_raw_mp4 = "videos-raw-mp4"
pose_2d = "pose-2d"
pose_2d_filter = "pose-2d-filtered"
pose_2d_projected = "pose-2d-proj"
pose_3d = "pose-3d"
pose_3d_filter = "pose-3d-filtered"
videos_labeled_2d = "videos-labeled"
videos_labeled_2d_filter = "videos-labeled-filtered"
calibration_videos = "calibration"
calibration_results = "calibration"
videos_labeled_3d = "videos-3d"
videos_labeled_3d_filter = "videos-3d-filtered"
angles = "angles"
summaries = "summaries"
videos_combined = "videos-combined"
videos_compare = "videos-compare"
videos_2d_projected = "videos-2d-proj"

[filter]
enabled = false

MOVE VIDEOS

In [ ]:
# set directories with video data 
calibration_path = r'E:\StopSignalSkinnerbox_local\calibration'
behavior_path = r'E:\StopSignalSkinnerbox_local\concatenated'

In [ ]:
# populate new project
copy = False # copy or move
list_of_sessions = os.listdir(projectpath)
calibration_videos = os.listdir(calibration_path)
behavior_videos = os.listdir(behavior_path)


for session in list_of_sessions:
    # check if directories are populated
    session_dirs = os.listdir(os.path.join(projectpath, session))
    
    # subset relevant videos 
    date = session[0:10] #YYYY_MM_DD_(conditions!!)_P007_camR/L.mp4 #session naming convention: 2022_04_25_P403
    id = session[11:]
    videos_to_move = []
    videos_moved = []
    session_calibration_videos = [video for video in calibration_videos if id in video if date in video]
    session_behavior_videos = [video for video in behavior_videos if id in video if date in video]

    for dir in session_dirs:
        done = os.listdir(os.path.join(projectpath, session, dir))
        
        # find videos for
        if dir == 'calibration':
            videos_to_move.extend([os.path.join(calibration_path,video) for video in session_calibration_videos if video not in done])
            videos_moved.extend([os.path.join(projectpath, session, dir, video) for video in session_calibration_videos if video not in done])        
        else:
            videos_to_move.extend([os.path.join(behavior_path, video) for video in session_behavior_videos if video not in done])
            videos_moved.extend([os.path.join(projectpath, session, dir, video) for video in session_behavior_videos if video not in done])  

    # move or copy
    print(f'Moving {len(videos_to_move)} videos for session {session}:')

    if copy:
        for video1, video2 in zip(videos_to_move, videos_moved):
            copy_command = f'copy {video1} {video2}'
            os.system(copy_command)
            print(f'{video2} coped!')
    else:
        for video1, video2 in zip(videos_to_move, videos_moved):
            os.rename(video1, video2)
            print(f'{video2} moved!')

UPDATE SETTINGS

In [5]:
# make quick changes to config.toml
def change_toml(edits, configfile):
    # read config
    myconfig = toml.load(configfile)
    # change config
    for key, value in edits.items():
        myconfig[key] = value
    # dump config
    with open(configfile, 'w') as f:
        new_toml_string = toml.dump(myconfig, f)
    print(new_toml_string)

# check video extension by content
def check_videotype(projectpath, dir):
    session = os.listdir(projectpath)[0]
    path = os.path.join(projectpath, session, dir)  
    videotype = os.listdir(path)[0].split('.')[-1]
    return videotype

In [6]:
# edit project specific anipose configurations
edits = {
    'project': projectpath, 
    'model_folder': 'C:/Users/myore/Desktop/MYOREHAB/vecinita/test_2d-vecinita-2025-05-16/dlc-models-pytorch/iteration-0/test_2dMay16-trainset95shuffle1/train',             # change path to deeplabcut project here
    'nesting': 1, 
    'video_extension': 'mp4',       # change video type here
    'pipeline': {
        'videos_raw': 'videos-raw',
        'pose_2d': 'pose-2d',
        'calibration_videos': 'calibration',
    },
    'calibration': {                # change calibration settings here
        'board_type': 'charuco', 
        'board_size': [10, 7], 
        'board_marker_bits': 4, 
        'board_marker_dict_number': 50, 
        'board_marker_length': 2.00, 
        'board_square_side_length': 2.70, 
        'animal_calibration': False, 
        'fisheye': False,
        }, 
    'manual_verification': {
        'manually_verify': False,
        },
    'labeling': {                   # change labeling settings here
        'scheme': [
            ['Wrist', 'Thumb0', 'Thumb1', 'Thumb2', 'Thumb3'],
            ['Wrist', 'Index0', 'Index1', 'Index2', 'Index3'],
            ['Wrist', 'Middle0', 'Middle1', 'Middle2', 'Middle3'],
            ['Wrist', 'Ring0', 'Ring1', 'Ring2', 'Ring3'],
            ['Wrist', 'Pinkie0', 'Pinkie1', 'Pinkie2', 'Pinkie3'] ]
            }, 
    }

In [7]:
change_toml(edits, configfile)

video_extension = "mp4"
converted_video_speed = 1
project = "C:/Users/myore/Desktop/MYOREHAB\\HandTracking"
model_folder = "C:/Users/myore/Desktop/MYOREHAB/vecinita/test_2d-vecinita-2025-05-16/dlc-models-pytorch/iteration-0/test_2dMay16-trainset95shuffle1/train"
nesting = 1

[calibration]
board_type = "charuco"
board_size = [ 10, 7,]
board_marker_bits = 4
board_marker_dict_number = 50
board_marker_length = 2.0
board_square_side_length = 2.7
animal_calibration = false
fisheye = false

[manual_verification]
manually_verify = false

[triangulation]
ransac = false
optim = false
scale_smooth = 2
scale_length = 2
scale_length_weak = 1
reproj_error_threshold = 5
score_threshold = 0.8
n_deriv_smooth = 3
constraints = []
constraints_weak = []

[pipeline]
videos_raw = "videos-raw"
pose_2d = "pose-2d"
calibration_videos = "calibration"

[filter]
enabled = false
type = "medfilt"
medfilt = 13
offset_threshold = 25
score_threshold = 0.05
spline = true
n_back = 5
multiprocessing = false

[filter3d]
e

CHECK CSV

In [2]:
import pandas as pd

In [36]:
df = pd.read_hdf("C:/Users/myore/Desktop/MYOREHAB/HandTracking/session1/pose-2d-filtered/camA.h5")
print(df.columns.levels[0])

Index(['DLC_resnet50_test_2dMay16shuffle2_100000'], dtype='object')


In [37]:
bodyparts_in_csv = df.columns.levels[1]
print(list(bodyparts_in_csv))

['Index0', 'Index1', 'Index2', 'Index3', 'Middle0', 'Middle1', 'Middle2', 'Middle3', 'Pinkie0', 'Pinkie1', 'Pinkie2', 'Pinkie3', 'Ring0', 'Ring1', 'Ring2', 'Ring3', 'Thumb0', 'Thumb1', 'Thumb2', 'Thumb3', 'Wrist']


In [38]:
for col in df.columns:
    print(col)

('DLC_resnet50_test_2dMay16shuffle2_100000', 'Wrist', 'x')
('DLC_resnet50_test_2dMay16shuffle2_100000', 'Wrist', 'y')
('DLC_resnet50_test_2dMay16shuffle2_100000', 'Wrist', 'likelihood')
('DLC_resnet50_test_2dMay16shuffle2_100000', 'Thumb0', 'x')
('DLC_resnet50_test_2dMay16shuffle2_100000', 'Thumb0', 'y')
('DLC_resnet50_test_2dMay16shuffle2_100000', 'Thumb0', 'likelihood')
('DLC_resnet50_test_2dMay16shuffle2_100000', 'Thumb1', 'x')
('DLC_resnet50_test_2dMay16shuffle2_100000', 'Thumb1', 'y')
('DLC_resnet50_test_2dMay16shuffle2_100000', 'Thumb1', 'likelihood')
('DLC_resnet50_test_2dMay16shuffle2_100000', 'Thumb2', 'x')
('DLC_resnet50_test_2dMay16shuffle2_100000', 'Thumb2', 'y')
('DLC_resnet50_test_2dMay16shuffle2_100000', 'Thumb2', 'likelihood')
('DLC_resnet50_test_2dMay16shuffle2_100000', 'Thumb3', 'x')
('DLC_resnet50_test_2dMay16shuffle2_100000', 'Thumb3', 'y')
('DLC_resnet50_test_2dMay16shuffle2_100000', 'Thumb3', 'likelihood')
('DLC_resnet50_test_2dMay16shuffle2_100000', 'Index0', 'x'

In [21]:
import pandas as pd

# Load the h5 file
h5_path = r"C:/Users/myore/Desktop/MYOREHAB/HandTracking/session1/pose-2d-filtered/camE.h5"
df = pd.read_hdf(h5_path)

# Rename all columns that start with "Thump" to "Thumb"
df.columns = pd.MultiIndex.from_tuples([
    (scorer, col.replace("Thump", "Thumb"), coord)
    if "Thump" in col else (scorer, col, coord)
    for scorer, col, coord in df.columns
])

# Save back to h5
df.to_hdf(h5_path, key='df')
print("✅ Renamed 'Thump*' to 'Thumb*' and saved.")


✅ Renamed 'Thump*' to 'Thumb*' and saved.


In [35]:
import pandas as pd
from pandas.io.pytables import HDFStore

h5_path = r"C:/Users/myore/Desktop/MYOREHAB/HandTracking/session1/pose-2d-filtered/camE.h5"

with HDFStore(h5_path) as store:
    print("Available keys in this .h5 file:")
    print(store.keys())


Available keys in this .h5 file:
['/df']


In [34]:
import pandas as pd

# Path to your h5 file
h5_path = r"C:/Users/myore/Desktop/MYOREHAB/HandTracking/session1/pose-2d-filtered/camE.h5"

# Load the correct dataset
df = pd.read_hdf(h5_path, key='df')

# Overwrite the file with only 'df' as the key
df.to_hdf(h5_path, key='df', mode='w')

print("✅ Cleaned .h5 file — only 'df' key remains.")


✅ Cleaned .h5 file — only 'df' key remains.


In [40]:
import cv2

# Path to your video
video_path = r"C:/Users/myore/Desktop/MYOREHAB/HandTracking/session1/videos-raw/camA.mp4"

# Frame number to extract (e.g., 100th frame)
frame_number = 1

# Open the video
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("❌ Error: Could not open video.")
else:
    # Set the frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    # Read the frame
    ret, frame = cap.read()
    if ret:
        # Save the frame as an image
        output_path = "C:/Users/myore/Desktop/MYOREHAB/extracted_frame.jpg"
        cv2.imwrite(output_path, frame)
        print(f"✅ Frame {frame_number} saved as {output_path}")
    else:
        print(f"❌ Error: Could not read frame {frame_number}")

    cap.release()


✅ Frame 1 saved as C:/Users/myore/Desktop/MYOREHAB/extracted_frame.jpg


In [44]:
import numpy as np

img = cv2.imread("C:/Users/myore/Desktop/MYOREHAB/extracted_frame.jpg")

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

print(f"Shape: {img.shape}, Dtype: {img.dtype}, Any NaNs? {np.isnan(img).any()}")

Shape: (540, 720, 3), Dtype: uint8, Any NaNs? False


In [53]:
import cv2
import numpy as np
import skvideo.io

def pad_to_even(img):
    h, w = img.shape[:2]
    pad_bottom = 0 if h % 2 == 0 else 1
    pad_right = 0 if w % 2 == 0 else 1
    if pad_bottom or pad_right:
        img = cv2.copyMakeBorder(img, 0, pad_bottom, 0, pad_right, cv2.BORDER_CONSTANT, value=(0,0,0))
    return img

frame = cv2.imread('C:/Users/myore/Desktop/MYOREHAB/extracted_frame.jpg')
if frame is None:
    raise RuntimeError("Failed to load image")

frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
frame = pad_to_even(frame)
print("Frame shape:", frame.shape)
print("Frame dtype:", frame.dtype)
print("Frame min/max:", frame.min(), frame.max())  # Should be (540, 720, 3)
frame = frame.astype(np.uint8)

writer = skvideo.io.FFmpegWriter(
    "test_output.mp4",
    inputdict={'-framerate': '60'},
    outputdict={
        #'-vcodec': 'libx264',
        #'-qp': '28',
        #'-pix_fmt': 'yuv420p',
        # Removed '-vf' line here
    }
)

for _ in range(10):
    writer.writeFrame(frame)

writer.close()
print("Test video written successfully")

Frame shape: (540, 720, 3)
Frame dtype: uint8
Frame min/max: 7 255
Test video written successfully


In [52]:
import cv2
import numpy as np

frame = cv2.imread('C:/Users/myore/Desktop/MYOREHAB/extracted_frame.jpg')
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# Convert RGB back to BGR because OpenCV VideoWriter expects BGR
frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # try mp4v for wide support
out = cv2.VideoWriter('test_opencv_output.mp4', fourcc, 60, (frame.shape[1], frame.shape[0]))

for _ in range(10):
    out.write(frame_bgr)

out.release()
print("OpenCV video written successfully")

OpenCV video written successfully


In [3]:
import os
print(os.listdir('C:/Users/myore/Desktop/MYOREHAB/HandTracking_mediapipe/recording/videos-raw'))

['camA.mp4', 'camB.mp4', 'camC.mp4', 'camD.mp4', 'camE.mp4']


In [ ]:
import cv2

cap = cv2.VideoCapture("C:/Users/myore/Desktop/MYOREHAB/HandTracking_mediapipe/recording/videos-raw/camA.mp4")

if not cap.isOpened():
    print(f"Error: Could not open video")
else:
    print(f"Video loaded successfully.")

Video loaded successfully.


In [11]:
files = os.listdir('C:/Users/myore/Desktop/MYOREHAB/HandTracking_mediapipe/recording/videos-raw')
print(files)
for file in files:
    VIDEO_FILE = file.replace('.mp4','')
    print('videos-raw/'+VIDEO_FILE+'.mp4')
    cap = cv2.VideoCapture('C:/Users/myore/Desktop/MYOREHAB/HandTracking_mediapipe/recording/videos-raw/'+VIDEO_FILE+'.mp4')
    
    if not cap.isOpened():
        print('Error:'+VIDEO_FILE+' not loaded.')
    else:
        print("Load successfully")    

['camA.mp4', 'camB.mp4', 'camC.mp4', 'camD.mp4', 'camE.mp4']
videos-raw/camA.mp4
Load successfully
videos-raw/camB.mp4
Load successfully
videos-raw/camC.mp4
Load successfully
videos-raw/camD.mp4
Load successfully
videos-raw/camE.mp4
Load successfully
